## Autoencoder

#### What: 
- a neural network that learns to predict its input data
- by encoding its input into a lower dimension representation space / vector layer
- then decoding the representation space / vector layer to get the output 
- model learns by minimising difference between input and output

#### So what: 
- achieves dimension reduction via non-linear mapping to a lower dimension space

#### How: 
- intuition: learning neural network with lower dimensions in the central hidden layers
- model architecture:
    - encoder (function)
    - decoder (inverse function)
    - output layer (depending on required output format)
- loss function: 
    - mse
    - cross entropy (?)


#### Why / Why not : 
- Deterministic representation 
- Extension: Variational autoencoding

##### Questions


- How does changing the layers result in differing results 
    - deeper layers - diff between going from [28*28 to 16] vs gradually.. - is this a matter of bias vs variance?
    - layers with lesser neurons - greater dimension reduction, results in more global/generalised learning? 
- (Optim) What is the adam optimiser: a method for stochastic optimisation

## Implementation
ref: https://www.youtube.com/watch?v=c36lUUr864M&t=2080s

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [21]:
transform = transforms.ToTensor() # to transform image into torch tensor

# transform = transforms.Compose([
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5), (0.5)) # output = (input - mean) / std
# ])

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# returns tuple (image, target class)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data, 
                                          batch_size=64, 
                                          shuffle=True)


In [9]:
iter(data_loader)
# dataloader class / object 
# <torch.utils.data.dataloader._SingleProcessDataLoaderIter at 0x16af6f910>

In [22]:
dataiter = iter(data_loader)
images, labels = next(dataiter)
print(torch.min(images), torch.max(images))
# to note the numerical range of image data 
# to use sigmoid function at the end: 
# set [min, max] = [0, 1]
# to use tanh: 
# set [min, max] = [-1, 1]

tensor(0.) tensor(1.)


In [29]:
class Autoencoder(nn.Module):
    def __init__(self):
        # ought to first initialise the super class
        super().__init__()
        # N (batch_size), 28*28
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128), 
            nn.ReLU(),
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12), 
            nn.ReLU(), 
            nn.Linear(12, 64), 
            nn.ReLU(), 
            nn.Linear(64, 128), 
            nn.ReLU(), 
            nn.Linear(128, 28*28),
            nn.Sigmoid() 
            # to correspond with input image range
        )

    # inference?
    def forward(self, x):
        encoder = self.encoder(x)
        decoded = self.decoder(encoder)
        return decoded


In [30]:
# Set up

model = Autoencoder()
# loss function = mse
criterion = nn.MSELoss()
# TODO: what is the adam optimiser: 
# a method for stochastic optimisation
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)
# weight decay - l2 regularisation on the neural network weights
# to keep nn weights small 


In [35]:
# Training loop 

num_epochs = 10
outputs = []

for epoch in range(num_epochs):
    for (img, _) in data_loader: 
        img = img.reshape(-1, 28*28) #784
        recon = model(img)
        loss = criterion(recon, img)

        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')

Epoch:1, Loss:0.0429
Epoch:2, Loss:0.0360
Epoch:3, Loss:0.0337
Epoch:4, Loss:0.0383
Epoch:5, Loss:0.0362
Epoch:6, Loss:0.0311
Epoch:7, Loss:0.0367
Epoch:8, Loss:0.0370
Epoch:9, Loss:0.0387
Epoch:10, Loss:0.0360


### Plot images 

### CNN Autoencoder